In [41]:
import pandas as pd
import numpy as np

data = pd.read_csv('../data/backtest_data_after_regime_prediction.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

## Add bollinger bands

In [42]:
# create a moving average
data['ema_20'] = data['Close'].ewm(span=20, adjust=False).mean()

In [43]:
# add bollinger bands 1SD from ema_20

# Calculate the standard deviation over the same span
data['stddev_20'] = data['Close'].ewm(span=20, adjust=False).std()

# Calculate the upper Bollinger Band 1SD
data['upper_band_1sd'] = data['ema_20'] + (data['stddev_20'] * 1)

# Calculate the lower Bollinger Band 1SD
data['lower_band_1sd'] = data['ema_20'] - (data['stddev_20'] * 1)

# Calculate the upper Bollinger Band 1.5SD
data['upper_band_1.5sd'] = data['ema_20'] + (data['stddev_20'] * 1.5)

# Calculate the lower Bollinger Band 1.5SD
data['lower_band_1.5sd'] = data['ema_20'] - (data['stddev_20'] * 1.5)

## Add fibonacci levels

In [44]:
# Create new columns for Fibonacci levels
data['fib_level_0.236'] = np.nan
data['fib_level_0.382'] = np.nan
data['fib_level_0.618'] = np.nan

# Calculate Fibonacci levels for each day
for i in range(251, len(data)):
    window = data.iloc[i-251:i+1]
    low = window['Low'].min()
    high = window['High'].max()
    diff = high - low
    data.loc[data.index[i], 'fib_level_0.236'] = high - 0.236 * diff
    data.loc[data.index[i], 'fib_level_0.382'] = high - 0.382 * diff
    data.loc[data.index[i], 'fib_level_0.618'] = high - 0.618 * diff

## Add MACD

In [47]:
# Define the short and long lookback periods
short_period = 12 #12 day ema
long_period = 26 #26 day ema

# Calculate the short and long period exponential moving averages
data['short_ema'] = data['Close'].ewm(span=short_period, adjust=False).mean()
data['long_ema'] = data['Close'].ewm(span=long_period, adjust=False).mean()

# Calculate the MACD line
data['MACD_line'] = data['short_ema'] - data['long_ema']

# Calculate the signal line
signal_period = 9
data['signal_line'] = data['MACD_line'].ewm(span=signal_period, adjust=False).mean()

# Calculate the MACD histogram
data['MACD_histogram'] = data['MACD_line'] - data['signal_line']

## Add RSI

In [49]:
# Define the lookback period
lookback_period = 14

# Calculate the difference in price
delta = data['Close'].diff()

# Separate positive and negative differences
up, down = delta.copy(), delta.copy()
up[up < 0] = 0
down[down > 0] = 0

# Calculate the average gain and average loss
avg_gain = up.ewm(com=lookback_period-1, min_periods=lookback_period).mean()
avg_loss = abs(down.ewm(com=lookback_period-1, min_periods=lookback_period).mean())

# Calculate RS
RS = avg_gain / avg_loss

# Calculate the RSI
data['RSI'] = 100 - (100 / (1 + RS))

In [50]:
data.tail()

,regime,Volume,Open,High,Low,Close,ema_20,stddev_20,upper_band_1sd,lower_band_1sd,...,lower_band_1.5sd,fib_level_0.236,fib_level_0.382,fib_level_0.618,short_ema,long_ema,MACD_line,signal_line,MACD_histogram,RSI
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-01,blue,4748110000,5098.51,5140.33,5094.16,5137.08,5020.372353,99.135149,5119.507501,4921.237204,...,4871.669630,4826.10308,4631.70846,4317.48154,5057.907372,4993.257677,64.649695,62.203764,2.445931,70.387006
2024-03-04,blue,4758440000,5130.99,5149.67,5127.18,5130.95,5030.903557,100.004308,5130.907865,4930.899249,...,4880.897096,4833.23884,4637.48058,4321.04942,5069.144699,5003.457108,65.687591,62.900530,2.787061,69.220177
2024-03-05,blue,4418410000,5110.52,5114.54,5056.82,5078.65,5035.450837,96.203826,5131.654663,4939.247011,...,4891.145099,4833.23884,4637.48058,4321.04942,5070.607053,5009.026952,61.580101,62.636444,-1.056343,60.070573
2024-03-06,blue,4559050000,5108.03,5127.97,5092.22,5104.76,5042.051710,93.858649,5135.910359,4948.193061,...,4901.263737,4833.23884,4637.48058,4321.04942,5075.861353,5016.118289,59.743064,62.057768,-2.314704,62.719905
2024-03-07,blue,4137980000,5132.38,5165.62,5128.21,5157.36,5053.033452,95.793725,5148.827177,4957.239727,...,4909.342865,4845.42464,4647.33768,4327.14232,5088.399606,5026.580638,61.818968,62.010008,-0.191040,67.411036


## Export file

In [51]:
data.to_csv('../data/backtest_data_with_technical_indicators.csv')